# Wine review Analysis

##  Import libraries

In [132]:
import pandas as pd
import numpy as np
import statistics 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df = pd.read_csv('winemag-data_first150k.csv')

In [3]:
df.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la Bégude


In [7]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150930 entries, 0 to 150929
Data columns (total 10 columns):
country        150925 non-null object
description    150930 non-null object
designation    105195 non-null object
points         150930 non-null int64
price          137235 non-null float64
province       150925 non-null object
region_1       125870 non-null object
region_2       60953 non-null object
variety        150930 non-null object
winery         150930 non-null object
dtypes: float64(1), int64(1), object(8)
memory usage: 11.5+ MB


In [129]:
df.shape

(150930, 10)

In [26]:
df.variety.value_counts()

Chardonnay                       14482
Pinot Noir                       14291
Cabernet Sauvignon               12800
Red Blend                        10062
Bordeaux-style Red Blend          7347
Sauvignon Blanc                   6320
Syrah                             5825
Riesling                          5524
Merlot                            5070
Zinfandel                         3799
Sangiovese                        3345
Malbec                            3208
White Blend                       2824
Rosé                              2817
Tempranillo                       2556
Nebbiolo                          2241
Portuguese Red                    2216
Sparkling Blend                   2004
Shiraz                            1970
Corvina, Rondinella, Molinara     1682
Rhône-style Red Blend             1505
Barbera                           1365
Pinot Gris                        1365
Cabernet Franc                    1363
Sangiovese Grosso                 1346
Pinot Grigio             

In [100]:
df.designation.value_counts()

Reserve                                        2752
Reserva                                        1810
Estate                                         1571
Barrel sample                                  1326
Riserva                                         754
Barrel Sample                                   639
Brut                                            624
Crianza                                         503
Estate Grown                                    449
Estate Bottled                                  396
Dry                                             374
Old Vine                                        331
Gran Reserva                                    330
Brut Rosé                                       248
Extra Dry                                       244
Vieilles Vignes                                 225
Bien Nacido Vineyard                            195
Rosé                                            180
Late Bottled Vintage                            171
Réserve     

In [66]:
vine_list= {}

for index, value in zip(df.variety.value_counts().index, df.variety.value_counts()):
    vine_list[index] = value

vine_list


{'Chardonnay': 14482,
 'Pinot Noir': 14291,
 'Cabernet Sauvignon': 12800,
 'Red Blend': 10062,
 'Bordeaux-style Red Blend': 7347,
 'Sauvignon Blanc': 6320,
 'Syrah': 5825,
 'Riesling': 5524,
 'Merlot': 5070,
 'Zinfandel': 3799,
 'Sangiovese': 3345,
 'Malbec': 3208,
 'White Blend': 2824,
 'Rosé': 2817,
 'Tempranillo': 2556,
 'Nebbiolo': 2241,
 'Portuguese Red': 2216,
 'Sparkling Blend': 2004,
 'Shiraz': 1970,
 'Corvina, Rondinella, Molinara': 1682,
 'Rhône-style Red Blend': 1505,
 'Barbera': 1365,
 'Pinot Gris': 1365,
 'Cabernet Franc': 1363,
 'Sangiovese Grosso': 1346,
 'Pinot Grigio': 1305,
 'Viognier': 1263,
 'Bordeaux-style White Blend': 1261,
 'Champagne Blend': 1238,
 'Port': 1058,
 'Grüner Veltliner': 1042,
 'Gewürztraminer': 982,
 'Portuguese White': 941,
 'Petite Sirah': 897,
 'Carmenère': 761,
 'Tempranillo Blend': 756,
 'Chenin Blanc': 746,
 'Glera': 622,
 'Grenache': 603,
 'Prosecco': 594,
 'Albariño': 537,
 'Gamay': 535,
 'Pinot Blanc': 470,
 "Nero d'Avola": 458,
 'Moscato'

In [98]:
vine_more_100 = []
for key, value in vine_list.items():
    if value > 100:
        vine_more_100.append(key)

In [101]:
vine_more_100

['Chardonnay',
 'Pinot Noir',
 'Cabernet Sauvignon',
 'Red Blend',
 'Bordeaux-style Red Blend',
 'Sauvignon Blanc',
 'Syrah',
 'Riesling',
 'Merlot',
 'Zinfandel',
 'Sangiovese',
 'Malbec',
 'White Blend',
 'Rosé',
 'Tempranillo',
 'Nebbiolo',
 'Portuguese Red',
 'Sparkling Blend',
 'Shiraz',
 'Corvina, Rondinella, Molinara',
 'Rhône-style Red Blend',
 'Barbera',
 'Pinot Gris',
 'Cabernet Franc',
 'Sangiovese Grosso',
 'Pinot Grigio',
 'Viognier',
 'Bordeaux-style White Blend',
 'Champagne Blend',
 'Port',
 'Grüner Veltliner',
 'Gewürztraminer',
 'Portuguese White',
 'Petite Sirah',
 'Carmenère',
 'Tempranillo Blend',
 'Chenin Blanc',
 'Glera',
 'Grenache',
 'Prosecco',
 'Albariño',
 'Gamay',
 'Pinot Blanc',
 "Nero d'Avola",
 'Moscato',
 'Rhône-style White Blend',
 'Garganega',
 'Garnacha',
 'Torrontés',
 'Dolcetto',
 'Montepulciano',
 'Aglianico',
 'Meritage',
 'Verdejo',
 'Cabernet Blend',
 'Primitivo',
 'Sauvignon',
 'Cabernet Sauvignon-Merlot',
 'Vermentino',
 'Blaufränkisch',
 'Ti

In [117]:
df.groupby(['country','variety'])['points'].mean()


country    variety                          
Albania    Kallmet                              88.000000
Argentina  Barbera                              85.000000
           Bonarda                              86.225166
           Bordeaux-style Red Blend             89.575472
           Bordeaux-style White Blend           83.000000
           Cabernet Blend                       89.750000
           Cabernet Franc                       87.454545
           Cabernet Sauvignon                   85.527745
           Cabernet Sauvignon-Cabernet Franc    84.000000
           Cabernet Sauvignon-Malbec            87.027027
           Cabernet Sauvignon-Merlot            87.136364
           Cabernet Sauvignon-Shiraz            87.250000
           Cabernet Sauvignon-Syrah             83.500000
           Cabernet-Syrah                       88.000000
           Champagne Blend                      84.000000
           Chardonnay                           84.177489
           Chardonnay-Sauvi

In [127]:
df_wine_100 = df[df.variety.isin(vine_more_100)]

In [128]:
df_wine_100.shape

(142934, 10)